**Programmer:** python_scripts (Abhijith Warrier)

**PYTHON SCRIPT TO _CUT PER-OBJECT MEMORY USING `__slots__`_. 🐍📦**

`__slots__` removes the per-instance `__dict__`, which can **dramatically reduce memory** and slightly **speed up attribute access** in object-heavy apps.
This notebook compares a normal class vs a slots class, shows attribute limitations, and benchmarks access speed.

## 📦 Import Standard Library

In [1]:
import tracemalloc          # measure memory usage by taking snapshots
import timeit               # quick micro-benchmarks for attribute access

## 📝 Snippet 1 — Normal Class vs `__slots__`: Memory Comparison

*We create many instances of a normal class and a slots class, and measure memory using `tracemalloc`.*
Note: `sys.getsizeof(obj)` won't include referenced objects; `tracemalloc` is better for this comparison.

In [2]:
# Define two equivalent classes: one normal, one with __slots__
class UserNormal:
    # regular class has a per-instance __dict__ (flexible but heavier)
    def __init__(self, uid, name, active):
        self.uid = uid                 # set attribute 'uid'
        self.name = name               # set attribute 'name'
        self.active = active           # set attribute 'active'

class UserSlots:
    __slots__ = ("uid", "name", "active")  # declare fixed attributes; no __dict__ by default
    def __init__(self, uid, name, active):
        self.uid = uid                      # set attribute 'uid'
        self.name = name                    # set attribute 'name'
        self.active = active                # set attribute 'active'

def make_users(cls, n=100_000):
    # helper to create n instances with small strings/ints
    return [cls(i, f"user{i}", (i % 2 == 0)) for i in range(n)]

# Measure memory for normal class instances
tracemalloc.start()                                     # begin tracking allocations
_ = make_users(UserNormal, n=100_000)                   # create many normal instances
snap_normal = tracemalloc.take_snapshot()               # snapshot memory after creation
tracemalloc.stop()                                      # stop tracking

# Measure memory for slots class instances
tracemalloc.start()                                     # restart tracking for a clean measurement
_ = make_users(UserSlots, n=100_000)                    # create many slots instances
snap_slots = tracemalloc.take_snapshot()                # snapshot memory after creation
tracemalloc.stop()                                      # stop tracking

# Compute total allocated size for each snapshot (sum all traces)
total_normal = sum(stat.size for stat in snap_normal.statistics('filename'))
total_slots  = sum(stat.size for stat in snap_slots.statistics('filename'))

print(f"Total allocated (normal): {total_normal/1024/1024:.2f} MiB")
print(f"Total allocated (slots) : {total_slots/1024/1024:.2f} MiB")
print(f"Memory reduction (~): {(1 - (total_slots/total_normal)) * 100:.1f}%")

Total allocated (normal): 17.73 MiB
Total allocated (slots) : 13.91 MiB
Memory reduction (~): 21.5%


## 🚧 Snippet 2 — Attribute Restrictions with `__slots__`

*Slots remove the per-instance `__dict__`, so you **cannot** add arbitrary attributes.*
This is a feature (saves memory, catches typos), but be aware of the limitation.

In [3]:
u = UserSlots(1, "alice", True)     # create a slots-based instance

try:
    u.email = "alice@example.com"   # ❌ adding new attribute not in __slots__ will fail
except AttributeError as e:
    print("Expected AttributeError:", e)

# If you need weak references, include "__weakref__" in __slots__
class UserSlotsWeak:
    __slots__ = ("uid", "name", "active", "__weakref__")  # allows weakref support if needed
    def __init__(self, uid, name, active):
        self.uid = uid
        self.name = name
        self.active = active

Expected AttributeError: 'UserSlots' object has no attribute 'email' and no __dict__ for setting new attributes


## ⏱️ Snippet 3 — Attribute Access Speed (Micro-benchmark)

*Access can be marginally faster with slots because lookups don’t go through `__dict__`.*
We benchmark simple attribute reads to illustrate.

In [4]:
# Prepare one instance of each for fair comparison
u_norm = UserNormal(1, "alice", True)
u_slots = UserSlots(1, "alice", True)

def read_norm():
    # read a couple of attributes; return something so it's not optimized away
    return u_norm.uid + (1 if u_norm.active else 0)

def read_slots():
    # same workload for the slots-based instance
    return u_slots.uid + (1 if u_slots.active else 0)

# timeit returns total seconds for the given number of loops; lower is better
t_norm  = timeit.timeit(read_norm, number=2_000_000)   # run many iterations
t_slots = timeit.timeit(read_slots, number=2_000_000)

print(f"Attribute read (normal): {t_norm:.3f}s")
print(f"Attribute read (slots) : {t_slots:.3f}s")
speedup = (t_norm / t_slots) if t_slots else float('inf')
print(f"Speedup (≈): {speedup:.2f}x")

Attribute read (normal): 0.086s
Attribute read (slots) : 0.058s
Speedup (≈): 1.47x


## ✅ One-liner Takeaway

**Use `__slots__` to shrink per-object memory (and often speed up attribute access) in object-heavy code — with the trade-off that you can’t add new attributes dynamically.**

### 📝Notes
- Include `"__weakref__"` in `__slots__` if you need weak references.
- With inheritance, all classes in the MRO should define compatible `__slots__`.
- `dataclasses.dataclass(slots=True)` (Python 3.10+) generates slots for you.